<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - StructType & StructField Explained with Examples
](https://sparkbyexamples.com/pyspark/pyspark-structtype-and-structfield/)

# 🏗️ PySpark StructType & StructField - הסבר עם דוגמאות

**תאריך:** 12 במאי, 2024  
**זמן קריאה משוער:** 18 דקות

---

## 🎯 מבוא

מחלקות **StructType** ו-**StructField** ב-PySpark משמשות לציון:
- 📋 ה-schema המותאם אישית ל-DataFrame
- 🏗️ יצירת עמודות מורכבות כמו nested struct, array ו-map columns

**StructType** הוא אוסף של אובייקטי **StructField** שמגדיר:
- שם עמודה
- טיפוס נתונים של עמודה
- בוליאני לציון אם העמודה יכולה להיות nullable או לא
- metadata

### 💡 נקודות מפתח:

- **StructType** הוא אוסף או רשימה של אובייקטי StructField
- **הגדרת DataFrame Schemas**: StructType משמש בדרך כלל להגדרת ה-schema בעת יצירת DataFrame, במיוחד עבור נתונים מובנים עם שדות מטיפוסי נתונים שונים
- **Nested Structures**: אתה יכול ליצור schemas מורכבים על ידי קינון StructType בתוך אובייקטי StructType אחרים, מה שמאפשר לך לייצג מבני נתונים היררכיים או מרובי רמות
- **Enforcing Data Structure**: כאשר קורא נתונים ממקורות שונים, ציון StructType כ-schema מבטיח שהנתונים מתפרשים ומובנים נכון. זה חשוב במיוחד בעת טיפול בנתונים חצי-מובנים או חסרי schema כמו קבצי JSON

## 📋 תוכן העניינים

1. שימוש ב-PySpark StructType & StructField עם DataFrame
2. הגדרת Nested StructType object struct
3. הוספה ושינוי struct של DataFrame
4. שימוש ב-SQL ArrayType ו-MapType
5. יצירת StructType object struct מקובץ JSON
6. יצירת StructType object struct ממחרוזת DDL
7. בדיקת קיום שדה ב-DataFrame
8. דוגמה מלאה

## 1️⃣ StructType - הגדרת מבנה ה-DataFrame

PySpark מספק מחלקת `StructType` מ-`pyspark.sql.types` כדי להגדיר את מבנה ה-DataFrame.

StructType מייצג schema, שהוא אוסף של אובייקטי **StructField** שמגדירים שם העמודה, טיפוס נתונים של העמודה, בוליאני לציון אם העמודה יכולה להיות nullable או לא, ו-metadata.

### 📐 מתודה printSchema()

מתודת `PySpark printSchema()` על DataFrame מציגה את StructType columns כ-"struct".

## 2️⃣ StructField - הגדרת metadata של DataFrame column

זה מייצג שדה ב-schema, המכיל metadata כמו:
- שם השדה
- טיפוס הנתונים שלו
- nullable של השדה

מחלקת StructField גם חלק מ-`pyspark.sql.types`.

## 3️⃣ שימוש ב-PySpark StructType & StructField עם DataFrame

בכל פעם שיוצרים DataFrame ב-PySpark, schema של DataFrame מסוק אוטומטית מהנתונים ומגדיר nullable ל-true לכל העמודות.

אנחנו יכולים לשנות את ההתנהגות הזו על ידי אספקת **schema** באמצעות **StructType** – שבו אנחנו יכולים לציין שם עמודה, טיפוס נתונים ו-nullable לכל field/column.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

data = [
    ("James", "", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "", "40288", "M", 4000),
    ("Robert", "", "Williams", "42114", "M", 4000),
    ("Maria", "Anne", "Jones", "39192", "F", 4000),
    ("Jen", "Mary", "Brown", "", "F", -1)
]

schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()
df.show(truncate=False)

**תוצאה צפויה:**

```
root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+
```

זה מייצר את ה-schema ותוצאת ה-DataFrame למטה.

## 4️⃣ הגדרת Nested StructType object struct

כדי להגדיר nested StructType ב-PySpark, השתמש ב-StructTypes פנימיים בתוך StructFields.

כל nested StructType הוא אוסף של StructFields, יוצר מבנה היררכי המייצג complex data בתוך DataFrame.

### 🏗️ יצירת Nested Structure

בדוגמה למטה, עמודת ה-**"name"** היא מסוג data StructType, המציינת מבנה מקונן.

המשמעות היא שעמודת ה-"name" עצמה מורכבת ממספר subfields או attributes, שנקראים multiple subfields.

In [ ]:
# הגדרת schema עם Nested Structure
structureData = [
    (("James", "", "Smith"), "36636", "M", 3000),
    (("Michael", "Rose", ""), "40288", "M", 4000),
    (("Robert", "", "Williams"), "42114", "M", 4000),
    (("Maria", "Anne", "Jones"), "39192", "F", 4000),
    (("Jen", "Mary", "Brown"), "", "F", -1)
]

structureSchema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("middlename", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

df2 = spark.createDataFrame(data=structureData, schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)  # הצג את כל העמודות

**Schema צפוי:**

```
root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
```

זה מייצר את ה-schema הבא. אם תשים לב, העמודה `name` היא מסוג struct המכילה את העמודות firstname, middlename, ו-lastname.

## 5️⃣ הוספה ושינוי struct של DataFrame

באמצעות **PySpark SQL function struct()**, אנחנו יכולים לשנות את ה-struct של ה-DataFrame הקיים ולהוסיף StructType חדש אליו.

הדוגמה למטה מדגימה איך להעתיק את העמודות ממבנה אחד לאחר ולהוסיף עמודה חדשה.

In [ ]:
# עדכון structtype קיים באמצעות struct
from pyspark.sql.functions import col, struct, when

updatedDF = df2.withColumn("OtherInfo",
    struct(
        col("id").alias("identifier"),
        col("gender").alias("gender"),
        col("salary").alias("salary"),
        when(col("salary").cast(IntegerType()) < 2000, "Low")
            .when(col("salary").cast(IntegerType()) < 4000, "Medium")
            .otherwise("High").alias("Salary_Grade")
    )).drop("id", "gender", "salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

**הסבר:**

כאן זה מעתיק את **"gender"**, **"salary"** ו-**"id"** ל-struct חדש **"OtherInfo"** ומוסיף עמודה חדשה **"Salary_Grade"**.

## 6️⃣ שימוש ב-SQL ArrayType ו-MapType

SQL StructType תומך גם ב-**ArrayType** ו-**MapType** כדי להגדיר את עמודות ה-DataFrame עבור:
- 📊 אוספי array
- 🗺️ אוספי map

בדוגמה למטה, עמודת `hobbies` מוגדרת כ-ArrayType(StringType) ו-`properties` מוגדרת כ-MapType(StringType, StringType) - כלומר שני key ו-value כ-String.

In [ ]:
from pyspark.sql.types import ArrayType, MapType

# שימוש ב-SQL ArrayType ו-MapType
arrayStructureSchema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("middlename", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("hobbies", ArrayType(StringType()), True),
    StructField("properties", MapType(StringType(), StringType()), True)
])

**הסבר על הטיפוסים:**

- **ArrayType(StringType())** - מערך של מחרוזות
- **MapType(StringType(), StringType())** - מיפוי מ-String ל-String (key-value pairs)

זה מייצר את ה-schema למטה. שים לב שהשדה `Hobbies` הוא מסוג array ו-`properties` הוא מסוג map.

## 7️⃣ יצירת StructType object struct מקובץ JSON

לחלופין, אתה יכול לטעון את ה-SQL StructType schema מקובץ JSON.

כדי להפוך את זה לפשוט, אקבל את ה-schema הנוכחי של DataFrame באמצעות:
- `df2.schema.json()`

אשמור את זה בקובץ, ואשתמש בו כדי ליצור schema מקובץ JSON זה.

In [ ]:
# שימוש ב-json() לטעינת StructType
print(df2.schema.json())

**תוצאה:**

זה מניב JSON format schema. אתה יכול לשמור זה בקובץ ולהשתמש בו כשנחוץ.

לחלופין, אתה יכול גם להשתמש ב-`df.schema.simpleString()`, זה יחזיר פורמט schema פשוט יותר.

### 📥 טעינת Schema מקובץ JSON

In [ ]:
# טעינת json schema ליצירת DataFrame
import json

schemaFromJson = StructType.fromJson(json.loads(schema.json()))
df3 = spark.createDataFrame(
    spark.sparkContext.parallelize(structureData),
    schemaFromJson
)
df3.printSchema()

## 8️⃣ יצירת StructType object struct ממחרוזת DDL

כדי ליצור אובייקט StructType, 'struct', מ-Data Definition Language (DDL) string ב-PySpark:

השתמש ב-**'StructType.fromDDL()'**. 

שיטה זו מפרסרת את מחרוזת ה-DDL ומייצרת אובייקט StructType שמשקף את ה-schema שהוגדר בה.

In [ ]:
# יצירת StructType מ-DDL String
ddlSchemaStr = """
    `fullName` STRUCT<`first`: STRING, `last`: STRING, `middle`: STRING>, 
    `age` INT, 
    `gender` STRING
"""

ddlSchema = StructType.fromDDL(ddlSchemaStr)
print(ddlSchema)

**הסבר:**

- `StructType.fromDDL()` - מפענח מחרוזת DDL ליצירת schema
- יצירת schema דינמי מ-external sources או configuration files

## 9️⃣ בדיקת קיום עמודה ב-DataFrame

כדי לבדוק אם עמודה קיימת ב-PySpark DataFrame:

השתמש בשיטת **'contains()'** על attribute ה-'columns' של DataFrame.

לדוגמה:
- `if "column_name" in df.columns`

בודק אם 'column_name' קיים ב-DataFrame 'df'.

לחלופין, אתה יכול להשתמש ב-**'selectExpr()'** עם column name ו-**'alias()'** כדי ליצור עמודה חדשה עם שם שונה, ואז לבדוק אם העמודה החדשה קיימת.

In [ ]:
# בדיקה אם עמודה קיימת
if "firstname" in df.columns:
    print("Column 'firstname' exists in the DataFrame.")
else:
    print("Column 'firstname' does not exist in the DataFrame.")

**חלופה:**

In [ ]:
# בדיקת קיום עמודה באמצעות contains()
print("firstname" in df.schema.fieldNames())
print(any(f.name == "firstname" and isinstance(f.dataType, StringType) for f in df.schema.fields))

**הדוגמה הזו מחזירה "true" לשני התרחישים.**

## 🎓 דוגמה מלאה של PySpark StructType & StructField

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType
from pyspark.sql.functions import col, struct, when

spark = SparkSession.builder.master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

data = [
    ("James", "", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "", "40288", "M", 4000),
    ("Robert", "", "Williams", "42114", "M", 4000),
    ("Maria", "Anne", "Jones", "39192", "F", 4000),
    ("Jen", "Mary", "Brown", "", "F", -1)
]

schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()
df.show(truncate=False)

In [ ]:
print("\n=== Nested Structure ===")
structureData = [
    (("James", "", "Smith"), "36636", "M", 3000),
    (("Michael", "Rose", ""), "40288", "M", 4000),
    (("Robert", "", "Williams"), "42114", "M", 4000),
    (("Maria", "Anne", "Jones"), "39192", "F", 4000),
    (("Jen", "Mary", "Brown"), "", "F", -1)
]

structureSchema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("middlename", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

df2 = spark.createDataFrame(data=structureData, schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

In [ ]:
print("\n=== עדכון Structure ===")
updatedDF = df2.withColumn("OtherInfo",
    struct(
        col("id").alias("identifier"),
        col("gender").alias("gender"),
        col("salary").alias("salary"),
        when(col("salary").cast(IntegerType()) < 2000, "Low")
            .when(col("salary").cast(IntegerType()) < 4000, "Medium")
            .otherwise("High").alias("Salary_Grade")
    )).drop("id", "gender", "salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

In [ ]:
print("\n=== Array & Map ===")
arrayStructureSchema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("middlename", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("hobbies", ArrayType(StringType()), True),
    StructField("properties", MapType(StringType(), StringType()), True)
])

**מצא דוגמה מלאה ב-GitHub project.**

## 📌 סיכום (Conclusion)

לסיכום, מחלקות **StructType** ו-**StructField** של PySpark מציעות כלים עוצמתיים להגדרה וניהול schemas של DataFrame.

**StructType** מספק גישה מובנית לארגון נתונים, מאפשר ציון מדויק של:
- שמות עמודות
- טיפוסי נתונים
- nullability

**StructField** מאפשר שליטה עדינה על שדות בודדים בתוך ה-schema, כולל:
- מבנים מקוננים
- טיפוסי נתונים מורכבים

### ✅ מה למדנו:

1. **יצירת Schema מפורש** - שליטה מלאה על מבנה הנתונים
2. **Nested Structures** - יצירת היררכיות מורכבות
3. **Array & Map Types** - עבודה עם אוספים
4. **Schema מ-JSON/DDL** - טעינה דינמית
5. **בדיקת קיום שדות** - validation

על ידי מינוף StructType ו-StructField, משתמשים יכולים:
- ✅ לבנות ולתפעל DataFrames ביעילות
- ✅ להבטיח עקביות נתונים
- ✅ לאפשר אינטגרציה חלקה עם מקורות נתונים מגוונים

**Happy Learning !!** 🎓

## 📚 מאמרים קשורים

- **PySpark Select Nested struct Columns**
- **PySpark Convert StructType (struct) to Dictionary/MapType (map)**
- **PySpark alias() Column & DataFrame Examples**
- **PySpark SparkContext Explained**
- **PySpark Check Column Exists in DataFrame**
- **PySpark Parse JSON from String Column | TEXT File**
- **PySpark MapType (Dict) Usage with Examples**
- **PySpark Convert DataFrame Columns to MapType (Dict)**
- **PySpark Create DataFrame From Dictionary (Dict)**

## 🏷️ תגיות (Tags)

`DATATYPE` `MAPTYPE` `PYSPARK.SCHEMA` `SCHEMA` `STRUCTFIELD` `STRUCTTYPE`

## 🔗 מקורות

**מקור המדריך:**
- SparkByExamples.com

**תיעוד רשמי:**
- [PySpark StructType Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructType.html)
- [PySpark StructField Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructField.html)

---

**💻 בהצלחה בלימוד PySpark!**